In [ ]:
# Install qubex library if not installed
# !pip install git+https://github.com/amachino/qubex.git

In [ ]:
import numpy as np
import qubex as qx

In [ ]:
# Create a flat-top pulse
flattop = qx.pulse.FlatTop(
    duration=30,
    amplitude=0.5,
    tau=10,
)

# Plot the pulse
flattop.plot()

# Print the pulse values
print(flattop.values)

In [ ]:
# Scale the amplitude of the pulse
flattop.scaled(1.5).plot()

# Shift the phase of the pulse
flattop.shifted(np.pi).plot()

# Repeat the pulse
flattop.repeated(3).plot()

In [ ]:
# Create a Gaussian pulse
gaussian = qx.pulse.Gaussian(
    duration=60,
    amplitude=0.5,
    sigma=10,
)

# Create a pulse array
arr = qx.PulseArray(
    [
        gaussian.shifted(np.pi / 2),
        qx.pulse.VirtualZ(np.pi / 4),
        gaussian.shifted(-np.pi / 2),
    ]
)

# Plot the pulse array
arr.plot()

# Print the pulse array values
print(arr.values)

In [ ]:
# Create an arbitrary pulse
iq_array = [
    0.1 + 0.2j,
    0.2 + 0.3j,
    0.3 + 0.4j,
    0.4 + 0.6j,
    0.5 + 0.6j,
]
arbit = qx.Pulse(iq_array)

# Plot the pulse
arbit.plot()

# Plot the reverse of the pulse
arbit.reversed().plot()

In [ ]:
# Define channel labels
labels = ["Q00", "Q01", "Q02"]

# Create a pulse schedule for the channels
with qx.PulseSchedule(labels) as seq1:
    seq1.add("Q00", flattop.repeated(5))
    seq1.add("Q01", arr.scaled(2))
    seq1.barrier()
    seq1.add("Q00", arbit.repeated(3))
    seq1.barrier(["Q00", "Q01"])
    seq1.add("Q01", qx.pulse.FlatTop(duration=100, amplitude=1, tau=10))
    seq1.add("Q02", qx.PulseArray([gaussian, qx.pulse.VirtualZ(np.pi / 2), gaussian]))

# Plot the pulse schedule
seq1.plot()

# Get the PulseArray dictionary of the channels
seq1.get_sequences()

In [ ]:
# Get the sampled sequences of the channels
seq1.get_sampled_sequences()

In [ ]:
# Create a pulse schedule using the previous pulse schedule

with qx.PulseSchedule(labels) as seq2:
    seq2.call(seq1)
    seq2.barrier()
    seq2.add("Q00", qx.pulse.Blank(100))
    seq2.barrier()
    seq2.call(seq1.reversed())

seq2.plot()

In [ ]:
# Example: Cross-Resonance Sequence

cr_drive = qx.pulse.FlatTop(
    duration=200,
    amplitude=1.0,
    tau=30,
)

drag_pi = qx.pulse.Drag(
    duration=10,
    amplitude=0.5,
    beta=0.3,
)

targets = {
    "Control": {"frequency": 5, "object": "Q00"},
    "CR": {"frequency": 6, "object": "Q00"},
    "Target": {"frequency": 6, "object": "Q01"},
}

with qx.PulseSchedule(targets) as cr:
    cr.add("CR", cr_drive)
    cr.add("Target", cr_drive.scaled(0.1).shifted(np.pi / 3))

with qx.PulseSchedule(targets) as ecr:
    ecr.call(cr)
    ecr.add("Control", drag_pi)
    ecr.call(cr.scaled(-1))

ecr.plot(title="Echoed Cross-Resonance")